# Couette flow example
This is an example of using the Couette flow
A two dimensional Couette flow is initialized and simulated. Afterwards the energy and the velocity field is plotted.

In [1]:
import lettuce as lt
import matplotlib.pyplot as plt
import numpy as np

ImportError: cannot import name 'AntiBounceBackOutlet' from partially initialized module 'lettuce' (most likely due to a circular import) (/home/philipp/lettuce/lettuce/__init__.py)

## Setup
* for running on GPU: device = "cuda". CUDA drivers are required!
* dtype=torch.float32 for single precision - float64 for double precision
* select collision model (here BGKCollision) - try also KBCCollision or RegularizedCollision

### Code:
* Reporters will grab the results in between simulation steps (see reporters.py and simulation.py)
* Output: Column 1: simulation steps, Column 2: time in LU, Column 3: kinetic energy in PU
* Output: separate VTK-file with ux,uy,(uz) and p for every 100. time step in ./output

In [2]:
nmax     = 200000
nconsole = 1000
nreport  = 100
epsilon  = 0.001 # convergence condition: .1 % relative change

context = lt.Context()
flow = lt.CouetteFlow2D(resolution=16, reynolds_number=100, mach_number=0.1,
                        context=context)
collision = lt.BGKCollision(tau=flow.units.relaxation_parameter_lu)
simulation = lt.Simulation(flow=flow, collision=collision, reporter=[])

Energy = lt.IncompressibleKineticEnergy(flow)
energy_reporter_internal = lt.ObservableReporter(Energy, interval=nreport, out=None)
simulation.reporter.append(energy_reporter_internal)
simulation.reporter.append(lt.ObservableReporter(Energy, interval=nconsole)) # print energy
simulation.reporter.append(lt.VTKReporter(interval=nreport, filename_base="./data/couette"))

NameError: name 'lt' is not defined

## Run simulation

In [3]:
energy_new = 0
mlups = 0
iterations = int(nmax//nconsole)
for _ in range(iterations):
    energy_old = energy_new
    energy_new = Energy(flow.f).mean()
    mlups += simulation(nconsole)
    if abs((energy_new - energy_old)/energy_new) < epsilon:
        print("CONVERGENCE! Less than ", epsilon*100, " % relative change")
        break
print("avg MLUPS: ", mlups/iterations)

NameError: name 'Energy' is not defined

## Post process
### Energy Reporter
* Grab output of kinetic energy reporter

In [4]:
energy = np.array(simulation.reporter[0].out)
print(energy.shape)
plt.plot(energy[:,1],energy[:,2])
plt.title('Kinetic energy')
plt.xlabel('Time')
plt.ylabel('Energy in physical units')
plt.show()

NameError: name 'np' is not defined

### Velocity
We calculate the speed in Lettuce units depending on the last 'f'. Then we convert this velocity into physical units. For further investigations the tensor must be converted into a Numpy-Array. The norm of the fractions in x and y direction is plotted afterwards.

In [5]:
u_x = context.convert_to_ndarray(flow.u_pu)[0]
plt.imshow(u_x.transpose())
plt.colorbar()
plt.show()

NameError: name 'context' is not defined